In [1]:
import httpx
import asyncio
import time

In [11]:
symbol = "TSLA"

async def download_basics():
    
    async with httpx.AsyncClient() as client:
        
        start = time.time()
        response = await client.get(f"https://www.tipranks.com/api/stocks/getData/?name={symbol}")
        json_data = response.json()
        
        return {
            "company_name": json_data["companyName"],
            "market": json_data["market"],
            "description": json_data["description"],
            "market_cap": json_data["marketCap"],
            "has_dividends": json_data["hasDividends"],
            "similar_stocks": [{"ticker": stock["ticker"], "company_name": stock["name"]} for stock in json_data["similarStocks"]],
            "yoy_change": json_data["momentum"]["momentum"] * 100,
        }

async def download_additionals():
    
    async with httpx.AsyncClient() as client:
        
        response = await client.get(f"https://market.tipranks.com/api/details/GetRealTimeQuotes/?tickers={symbol}&break=1582103013582")
        json_data = response.json()
        
        return {
            "year_low": json_data[0]["yLow"],
            "year_high": json_data[0]["yHigh"],
            "pe_ratio": json_data[0]["pe"],
            "market_cap": json_data[0]["marketCap"],
            "eps": json_data[0]["eps"],
        }
    
async def download_address_and_ipo():
    
    async with httpx.AsyncClient() as client:
        
        response = await client.get(f"https://finnhub.io/api/v1/stock/profile?symbol={symbol}")
        json_data = response.json()
        
        return {
            "address": json_data["address"],
            "city": json_data["city"],
            "country": json_data["country"],
            "website": json_data["weburl"] if json_data["weburl"].startswith("https") else "https://" + json_data["weburl"],
            "ipo": json_data["ipo"],
        }
    
basic, additionals, address_and_ipo = await asyncio.gather(download_basics(), download_additionals(), download_address_and_ipo())
basic.update(additionals)
basic.update(address_and_ipo)

basic

{'company_name': 'Tesla',
 'market': 'NASDAQ',
 'description': 'Tesla Motors,inc., designs, develops, manufactures and sells high-performance fully electric vehicles and advanced electric vehicle powertrain components.',
 'market_cap': '166.37B',
 'has_dividends': False,
 'similar_stocks': [{'ticker': 'DDAIF', 'company_name': 'Daimler'},
  {'ticker': 'GM', 'company_name': 'General Motors'},
  {'ticker': 'F', 'company_name': 'Ford Motor'},
  {'ticker': 'HMC', 'company_name': 'Honda Motor Company'},
  {'ticker': 'TM', 'company_name': 'Toyota Motor'},
  {'ticker': 'BAMXF',
   'company_name': 'Bayerische Motoren Werke Aktiengesellschaft'},
  {'ticker': 'NSANY', 'company_name': 'Nissan'}],
 'yoy_change': 203.21919619249073,
 'year_low': 177.0,
 'year_high': 968.99,
 'pe_ratio': 0.0,
 'eps': -5.01,
 'address': '3500 Deer Creek Road',
 'city': 'Palo Alto',
 'country': 'USA',
 'website': 'https://www.tesla.com',
 'ipo': '2010-06-29'}